#### `sparql_dataframe` do not provide the possibilty to set the userAgent. Therefore, the file `queryWikidata` slightly modifies the code of the library in order to set the userAgent, required by Wikidata in order to send requests t its sparql endpoint

In [16]:
#import sparql_dataframe
from pandas import *
from queryWikidata import query_wikidata

In [17]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

## The query used for the paper

In [18]:
paper_query = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  VALUES ?citizen {wd:Q38 wd:Q172579}
  VALUES ?role {wd:Q6625963 wd:Q49757}
  ?writer wdt:P345 ?imdb ;
          rdfs:label ?writerLabel ;
          (wdt:P106 | p:P106 / ps:P106) ?role ;
          (wdt:P27 | p:P27 / ps:P27) ?citizen.
  FILTER (lang(?writerLabel) = 'en') 
}
GROUP BY ?writer ?writerLabel ?imdb 
"""

In [19]:
paper_results = query_wikidata(endpoint, paper_query, user_agent, post=True).fillna("")
paper_results

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q193018,Gianni Rodari,nm0734427
1,http://www.wikidata.org/entity/Q3751286,Franco Enna,nm0257850
2,http://www.wikidata.org/entity/Q3767161,Giovanni Drovetti,nm0238144
3,http://www.wikidata.org/entity/Q527124,Vincenzo De Crescenzo,nm1876866
4,http://www.wikidata.org/entity/Q3791865,Idolina Landolfi,nm2285763
...,...,...,...
227,http://www.wikidata.org/entity/Q919103,Ugo Betti,nm0079348
228,http://www.wikidata.org/entity/Q1638370,Giorgio Caproni,nm2516736
229,http://www.wikidata.org/entity/Q587389,Lasse Braun,nm0105808
230,http://www.wikidata.org/entity/Q469656,Paolo Giordano,nm1279468


____________________


## `writers_query`
It retrieves every writer, prosaist, novelist or poet that has the Italian citizenship or lived in a country or historical country whose capital city is now an Italian city (to include also writers living, for example, in the Republic of Florence).
Writers are novelists, poets, prosaists or common "writers". However, when an entity is also a screenwriter, it has to be at least also a novelist or a poet. We restrict the field to authors born after 1200

Problems:
1. Is it better to restrict the threshold of being at least a poet or a novelist as well as a screenwriter excluding the poet possibility (an author to be included should be a novelist if he/she is a screewriter)?

In [29]:
wq_post_1200 = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  #get writers having an IMDb ID and a Date of Born
  ?writer wdt:P345 ?imdb ;
          wdt:P569 ?dob .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  FILTER ("1200-01-01"^^xsd:dateTime <= ?dob) 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?writer ?writerLabel ?imdb
"""

In [23]:
writers_post_1200_df = query_wikidata(endpoint, wq_post_1200, user_agent, True).fillna("")
writers_post_1200_df

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q3645969,Bruno Zanin,nm0953025
1,http://www.wikidata.org/entity/Q558097,Mario Rigoni Stern,nm1167259
2,http://www.wikidata.org/entity/Q1067,Dante Alighieri,nm0019604
3,http://www.wikidata.org/entity/Q3608591,Alberto Spadolini,nm0816553
4,http://www.wikidata.org/entity/Q3615513,Andrea Beltramo,nm1244099
...,...,...,...
718,http://www.wikidata.org/entity/Q12113,Hugo Pratt,nm0695463
719,http://www.wikidata.org/entity/Q1042535,Carlo Terron,nm1923000
720,http://www.wikidata.org/entity/Q378674,Arnoldo Foà,nm0289450
721,http://www.wikidata.org/entity/Q993477,Bruno Lauzi,nm0487336


## Query for work connections in wikidata

The first query in the first block uses only the IDs of the audiovisual works to retrieve data.

The second uses also the IDs of the writers

In [30]:
work_connections_query = """ 
SELECT DISTINCT ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
WHERE {
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  
  
  VALUES ?workType {wd:Q7725634 wd:Q47461344 wd:Q571 wd:Q3331189}   #define what are the possible types of writtenWrok
  ?writtenWork wdt:P50 ?writer ;                #get written wroks written by our writers having the types defined above
               wdt:P31 ?workType .
  
  ?audioVisualWork wdt:P345 ?imdbAudioVisual .  #define audioVisual work as everyting having an imdb ID y now (so to grasp as many products as possile as first)
  {                                             #get aV works derived from the works written by our writers, or inspired by them
    VALUES ?derivedBy {wdt:P737 wdt:P144 wdt:P8371}
    ?audioVisualWork ?derivedBy ?writtenWork .    
  }UNION{
    ?writtenWork wdt:P4969 ?audioVisualWork .    
  }
  
  
  
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
"""

In [26]:
work_connections_query_writers_id = """
SELECT DISTINCT ?audioVisualWork ?audioVisualWorkLabel  ?imdbAudioVisual
WHERE {
  #get writers having an IMDb ID
  ?writer wdt:P345 ?imdb .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  
  
  VALUES ?workType {wd:Q7725634 wd:Q47461344}   #define what are the possible types of writtenWrok
  ?writtenWork wdt:P50 ?writer ;                #get written wroks written by our writers having the types defined above
               wdt:P31 ?workType .
  
  ?audioVisualWork wdt:P345 ?imdbAudioVisual .  #define audioVisual work as everyting having an imdb ID y now (so to grasp as many products as possile as first)
  {                                             #get aV works derived from the works written by our writers, or inspired by them
    VALUES ?derivedBy {wdt:P737 wdt:P144 wdt:P8371}
    ?audioVisualWork ?derivedBy ?writtenWork .    
  }UNION{
    ?writtenWork wdt:P4969 ?audioVisualWork .    
  }
  
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?audioVisualWork ?audioVisualWorkLabel ?imdbAudioVisual
"""

In [31]:
work_connections_df = query_wikidata(endpoint, work_connections_query, user_agent, post= True).fillna("")
work_connections_df

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q967661,Pinocchio 3000,tt0332318
1,http://www.wikidata.org/entity/Q1425815,Captain Blood,tt0026174
2,http://www.wikidata.org/entity/Q18429,Fellini's Casanova,tt0074291
3,http://www.wikidata.org/entity/Q834460,The End Is My Beginning,tt1529292
4,http://www.wikidata.org/entity/Q683014,The Garden of the Finzi-Continis,tt0065777
...,...,...,...
154,http://www.wikidata.org/entity/Q5638667,Pinocchio,tt0818692
155,http://www.wikidata.org/entity/Q31185402,Filomena Marturano,tt0471700
156,http://www.wikidata.org/entity/Q1130910,Dante's Inferno,tt0026262
157,http://www.wikidata.org/entity/Q3791184,I promessi sposi,tt0172043


In [27]:
work_connections_query_wid_df = query_wikidata(endpoint, work_connections_query_writers_id, user_agent, post= True).fillna("")
work_connections_query_wid_df

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q114725240,The Hummingbird,tt14589916
1,http://www.wikidata.org/entity/Q1077585,The Truce,tt0117959
2,http://www.wikidata.org/entity/Q3760834,Geppetto,tt0200656
3,http://www.wikidata.org/entity/Q28214950,La fameuse invasion des ours en Sicile,tt6284064
4,http://www.wikidata.org/entity/Q48967415,The Name of the Rose,tt7572868
...,...,...,...
132,http://www.wikidata.org/entity/Q3792653,Il Decamerone,tt0167877
133,http://www.wikidata.org/entity/Q18206120,Corruption in the Palace of Justice,tt4103296
134,http://www.wikidata.org/entity/Q3665312,Caesar the Conqueror,tt0057105
135,http://www.wikidata.org/entity/Q1024861,Cabiria,tt0003740


In [34]:
concat_dfs = concat([work_connections_df, work_connections_query_wid_df])
concat_dfs.drop_duplicates()
concat_dfs

,audioVisualWork,audioVisualWorkLabel,imdbAudioVisual
0,http://www.wikidata.org/entity/Q967661,Pinocchio 3000,tt0332318
1,http://www.wikidata.org/entity/Q1425815,Captain Blood,tt0026174
2,http://www.wikidata.org/entity/Q18429,Fellini's Casanova,tt0074291
3,http://www.wikidata.org/entity/Q834460,The End Is My Beginning,tt1529292
4,http://www.wikidata.org/entity/Q683014,The Garden of the Finzi-Continis,tt0065777
...,...,...,...
132,http://www.wikidata.org/entity/Q3792653,Il Decamerone,tt0167877
133,http://www.wikidata.org/entity/Q18206120,Corruption in the Palace of Justice,tt4103296
134,http://www.wikidata.org/entity/Q3665312,Caesar the Conqueror,tt0057105
135,http://www.wikidata.org/entity/Q1024861,Cabiria,tt0003740


#### They do not include different works!

In [24]:
q1 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .
  VALUES ?occupation {wd:Q36180 wd:Q6625963 wd:Q49757 wd:Q12144794}
  ?writer wdt:P106 ?occupation
    
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [25]:
exe_q1 = query_wikidata(endpoint, q1, user_agent, True).fillna("")
exe_q1

,writer,writerLabel
0,http://www.wikidata.org/entity/Q120553,Paul Videsott
1,http://www.wikidata.org/entity/Q153700,Oriana Fallaci
2,http://www.wikidata.org/entity/Q161933,Alberto Moravia
3,http://www.wikidata.org/entity/Q165351,Mario Toso
4,http://www.wikidata.org/entity/Q175778,Corrado Balducci
...,...,...
7957,http://www.wikidata.org/entity/Q114568860,Fabio Bolzetta
7958,http://www.wikidata.org/entity/Q114591174,Matteo Giovanni Brega
7959,http://www.wikidata.org/entity/Q114598028,Sandro Bottega
7960,http://www.wikidata.org/entity/Q114442618,Giuseppe Giordano


_______________________


# Queries by progressively filtering results

In [35]:
qf_0 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      Values ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?country .                      #se fermo la query qui va in timeout
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
df0 = query_wikidata(endpoint, qf_0, user_agent, True).fillna("")
df0

In [9]:
qf_1 = """
SELECT DISTINCT ?writer ?writerLabel 
WHERE {
    {
      ?country wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?country wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }UNION{
      Values ?country{wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118}
    }
  ?writer (wdt:P27 | p:P27 / ps:P27) ?country .                      #se fermo la query qui va in timeout
  VALUES ?occupation {wd:Q36180 wd:Q6625963 wd:Q49757 wd:Q12144794}
  ?writer wdt:P106 ?occupation .                                       #se specifico i vari ruoli da scrittore senza eccezioni i risultati sono 9788
  
  
  { VALUES ?author {wdt:P50 wdt:P170}
    ?something wdt:P50 ?writer . #Qua ho messo solo P perchè se inserisco entrambe le possiblità sopra crea errori
   }UNION{
    VALUES ?authorOf {wdt:P1455 wdt:P800}
    ?writer ?authorOf ?something .
  }                                                                    #se chiedo tutti gli scrittori con almeno un opera su wikidata trovo 1633 risultati
  
  ?writer wdt:P345 ?imdbId .                                            #aggiungendo questa riga arriva a 235 risulati                               
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

#### 235 risultati
Interessante:
se mantengo la ricerca per autori che hanno un ID su IMDb ma rimuovo il fatto che debbano essre considerati come autori di almeno un opera su wikidta:

`  { VALUES ?author {wdt:P50 wdt:P170}
    ?something wdt:P50 ?writer .
   }UNION{
    VALUES ?authorOf {wdt:P1455 wdt:P800}
    ?writer ?authorOf ?something .
  } `

  Ottengo 1169 risultati


In [14]:
ex_qfilter = sparql_dataframe.get(endpoint, q_filter, post=True)

EndPointNotFound: EndPointNotFound: it was impossible to connect with the endpoint in that address, check if it is correct. 

Response:
b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>404 Not Found</title>\n</head><body>\n<h1>Not Found</h1>\n<p>The requested URL was not found on this server.</p>\n<hr>\n<address>Apache Server at query.wikidata.org Port 80</address>\n</body></html>\n'